## OpenVINO optimizations for Knowledge graphs 

The goal of this notebook is to showcase performance optimizations for the ConvE knowledge graph embeddings model using the Intel® Distribution of OpenVINO™ Toolkit.
The optimizations process contains the following steps:
1. Export the trained model to OpenVINO intermediate representation (IR)
2. Compare the inference performance with the optimized OpenVINO model

The ConvE model we use is an implementation of the paper Convolutional 2D Knowledge Graph Embeddings (https://arxiv.org/abs/1707.01476). The sample dataset was downloaded from: https://github.com/TimDettmers/ConvE/tree/master/countries/countries_S1

## Windows specific settings

In [ ]:
# On Windows, add the directory that contains cl.exe to the PATH to enable PyTorch to find the
# required C++ tools. This code assumes that Visual Studio 2019 is installed in the default
# directory. If you have a different C++ compiler, please add the correct path to os.environ["PATH"]
# directly. Note that the C++ Redistributable is not enough to run this notebook.

# Adding the path to os.environ["LIB"] is not always required - it depends on the system's configuration

import sys

if sys.platform == "win32":
    import distutils.command.build_ext
    import os
    from pathlib import Path

    VS_INSTALL_DIR = r"C:/Program Files (x86)/Microsoft Visual Studio"
    cl_paths = sorted(list(Path(VS_INSTALL_DIR).glob("**/Hostx86/x64/cl.exe")))
    if len(cl_paths) == 0:
        raise ValueError(
            "Cannot find Visual Studio. This notebook requires a C++ compiler. If you installed "
            "a C++ compiler, please add the directory that contains cl.exe to `os.environ['PATH']`."
        )
    else:
        # If multiple versions of MSVC are installed, get the most recent version
        cl_path = cl_paths[-1]
        vs_dir = str(cl_path.parent)
        os.environ["PATH"] += f"{os.pathsep}{vs_dir}"
        # Code for finding the library dirs from
        # https://stackoverflow.com/questions/47423246/get-pythons-lib-path
        d = distutils.core.Distribution()
        b = distutils.command.build_ext.build_ext(d)
        b.finalize_options()
        os.environ["LIB"] = os.pathsep.join(b.library_dirs)
        print(f"Added {vs_dir} to PATH")

## Import the packages needed for successful execution

In [ ]:
import numpy as np 
import time
import math
import json
import os

import torch 
from torch.nn import functional as F, Parameter
from torch.nn.init import xavier_normal_

from tqdm import tqdm
from pathlib import Path

from sklearn.metrics import accuracy_score

from openvino.runtime import Core

### Settings: Including path to the serialized model files and input data files



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

modelpath = Path('models/conve.pt')                                       # Path to the trained model

entdatapath = Path('data/countries_S1/kg_training_entids.txt')            # Path to the file containing the entities and entity IDs
reldatapath = Path('data/countries_S1/kg_training_relids.txt')            # Path to the file containing the relations and relation IDs
testdatapath = Path('data/countries_S1/e1rel_to_e2_ranking_test.json')    # Path to the test data file

batch_size = 1                                                            # Batch size
emb_dim = 300                                                             # Entity and relation embedding dimensions

input_dropout = 0.2                                                       # Input dropout for the model
dropout = 0.3                                                             # Dropout for the model
feature_map_dropout = 0.2                                                 # Feature map dropout for the model
use_bias = True 
top_k = 2                                                                 # Top K vals to consider from the predictions


### Required for OpenVINO conversion
output_dir = Path("models")
base_model_name = "conve"

output_dir.mkdir(exist_ok=True)

# Paths where PyTorch, ONNX and OpenVINO IR models will be stored
fp32_onnx_path = Path(output_dir / (base_model_name + "_fp32")).with_suffix(".onnx")
fp32_ir_path = fp32_onnx_path.with_suffix(".xml")

### Defining the ConvE model class

In [ ]:
# Model implementation reference: https://github.com/TimDettmers/ConvE
class ConvE(torch.nn.Module):
    def __init__(self, num_entities, num_relations):
        super(ConvE, self).__init__()
        # Embedding tables for entity and relations with num_uniq_ent in y-dim, emb_dim in x-dim
        self.emb_e = torch.nn.Embedding(num_entities, emb_dim, padding_idx=0)
        self.ent_weights_matrix = torch.ones([num_entities, emb_dim], dtype=torch.float64)
        self.emb_rel = torch.nn.Embedding(num_relations, emb_dim, padding_idx=0)
        self.ne = num_entities
        self.nr = num_relations
        self.inp_drop = torch.nn.Dropout(input_dropout)
        self.hidden_drop = torch.nn.Dropout(dropout)
        self.feature_map_drop = torch.nn.Dropout2d(feature_map_dropout)
        self.loss = torch.nn.BCELoss()

        self.conv1 = torch.nn.Conv2d(1, 32, (3, 3), 1, 0, bias=use_bias)
        self.bn0 = torch.nn.BatchNorm2d(1)
        self.bn1 = torch.nn.BatchNorm2d(32)
        self.ln0 = torch.nn.LayerNorm(emb_dim)
        self.register_parameter('b', Parameter(torch.zeros(num_entities)))
        self.fc = torch.nn.Linear(16128,emb_dim)
    
    def init(self):
        # Xavier initialization
        xavier_normal_(self.emb_e.weight.data)
        xavier_normal_(self.emb_rel.weight.data)
        
    
    def forward(self, e1, rel):
        e1_embedded= self.emb_e(e1).view(-1, 1, 10, 30)
        rel_embedded = self.emb_rel(rel).view(-1, 1, 10, 30)
        stacked_inputs = torch.cat([e1_embedded, rel_embedded], 2)

        stacked_inputs = self.bn0(stacked_inputs)
        x= self.inp_drop(stacked_inputs)
        x= self.conv1(x)
        x= self.bn1(x)
        x= F.relu(x)
        x = self.feature_map_drop(x)
        x = x.view(batch_size, -1)
        x = self.fc(x)
        x = self.hidden_drop(x)
        x = self.ln0(x)
        x = F.relu(x)
        x = torch.mm(x, self.emb_e.weight.transpose(1,0)) 
        x = self.hidden_drop(x)
        x += self.b.expand_as(x)
        pred = torch.nn.functional.softmax(x, dim=1)
        return pred

### Defining the dataloader

In [ ]:
class DataLoader():
    def __init__(self):
        super(DataLoader, self).__init__()
        
        self.ent_path = entdatapath
        self.rel_path = reldatapath
        self.test_file = testdatapath
        self.entity_ids = self.load_data(self.ent_path) 
        self.rel_ids =  self.load_data(self.rel_path)
        self.test_triples_list = self.convert_triples(self.test_file)


    def load_data(self, data_path):
        item_dict = {}
        with open(data_path) as df:
            lines = df.readlines()
            for line in lines:
                name, id = line.strip().split('\t')
                item_dict[name] = int(id)
        return item_dict
        
    def convert_triples(self, data_path):
        triples_list = []
        with open(data_path) as df:
            lines = df.readlines()
            for line in lines:
                item_dict = json.loads(line.strip())
                h = item_dict['e1']
                r = item_dict['rel']
                t = item_dict['e2_multi1'].split('\t')
                hrt_list = []
                hrt_list.append(self.entity_ids[h])
                hrt_list.append(self.rel_ids[r])
                t_ents = []
                for t_idx in t:
                    t_ents.append(self.entity_ids[t_idx])
                hrt_list.append(t_ents)
                triples_list.append(hrt_list)
        return triples_list

### Evaluate the trained ConvE model
We will first evaluate the model performance using PyTorch. The goal is to make sure there are no accuracy differences between the original model inference and the model converted to OpenVINO intermediate representation inference results. 
Here, we use a simple accuracy metric to evaluate the model performance. However, it is typical to use metrics such as Mean Reciprocal Rank, Hits@10 etc.

In [ ]:
data = DataLoader()
num_entities =  len(data.entity_ids)
num_relations =  len(data.rel_ids)

model = ConvE(num_entities, num_relations)
model.load_state_dict(torch.load(modelpath))
model.eval()

pt_inf_times = []

triples_list = data.test_triples_list
num_test_samples = len(triples_list)
pt_acc = 0.0
for i in range(num_test_samples):
    test_sample = triples_list[i]
    h,r,t = test_sample
    start_time = time.time()
    logits = model.forward(torch.tensor(h), torch.tensor(r))
    end_time = time.time()
    pt_inf_times.append(end_time-start_time)
    score, pred = torch.topk(logits, top_k, 1)
    
    gt = np.array(sorted(t))
    pred = np.array(sorted(pred[0].cpu().detach()))
    pt_acc += accuracy_score(gt, pred)

avg_pt_time = np.mean(pt_inf_times)*1000
print(f'Average time taken for inference: {avg_pt_time} ms')
print(f'Mean accuracy of the model on the test dataset: {pt_acc/num_test_samples}')


### Convert the trained PyTorch model to ONNX format
To evaluate performance with OpenVINO, we can either convert the trained PyTorch model to an intermediate representation (IR) format or to an ONNX representation. In this notebook, we use the ONNX format.
For more details on model optimization, refer to: https://docs.openvino.ai/latest/openvino_docs_MO_DG_Deep_Learning_Model_Optimizer_DevGuide.html 

In [ ]:
print(f'Converting the trained conve model to ONNX format')
torch.onnx.export(model, (torch.tensor(1), torch.tensor(1)), fp32_onnx_path, verbose=False, opset_version=11, training=False)

### Evaluate the model performance with OpenVINO
Now, we evaluate the model performance with the OpenVINO framework. In order to do so, we make three main API calls:
1. Initialize the Inference engine with Core()
2. Load the model with read_model()
3. Compile the model with compile_model()

The model can then be inferred on using by using the create_infer_request() API call. 

In [ ]:
ie = Core()
ir_net = ie.read_model(model=fp32_onnx_path)
compiled_model = ie.compile_model(model=ir_net)
output_layer = compiled_model.output(0)
request = compiled_model.create_infer_request()

ov_acc = 0.0
ov_inf_times = []
for i in range(num_test_samples):
    test_sample = triples_list[i]
    h,r,t = test_sample
    start_time = time.time()
    request.infer(inputs={'input.1': h,'input.2': r})
    end_time = time.time()
    result = request.get_output_tensor(output_layer.index).data
    ov_inf_times.append(end_time-start_time)
    top_k_idx = list(np.argpartition(result[0], -top_k)[-top_k:])
    
    gt = np.array(sorted(t))
    pred = np.array(sorted(top_k_idx))
    ov_acc += accuracy_score(gt, pred)


avg_ov_time = np.mean(ov_inf_times)*1000
print(f'Average time taken for inference: {avg_ov_time} ms')
print(f'Mean accuracy of the model on the test dataset: {ov_acc/num_test_samples}')

### Finally, we print the platform specific speedup obtained through OpenVINO graph optimizations

In [ ]:
print(f'Speedup with OpenVINO optimizations: {round(float(avg_pt_time)/float(avg_ov_time),2)} X')

### References
1. Convolutional 2D Knowledge Graph Embeddings, Tim Dettmers et al. (https://arxiv.org/abs/1707.01476)
2. Model implementation: https://github.com/TimDettmers/ConvE